In [2]:
# ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in 
# emails, as well as in banner ads on Facebook, Twitter, and Google. 
# They want to know how the two ads are performing on each of the different platforms on each day of the week.

In [2]:
import pandas as pd

In [27]:
# load the dataframe CSV and incpect it
ad_clicks = pd.read_csv('Micro project in Python- AB Testing with Pandads_Dataset.csv')
# Inspect the data 
print(ad_clicks.head(3))

# Remove the numbers from days
r_n = lambda x: x.split(' ')[-1]
ad_clicks['day'] = ad_clicks.day.apply(r_n)
print(ad_clicks.head(3))

# see the visits coming from email 
email = ad_clicks[ad_clicks.utm_source == 'email']
print(email.head(3))

# see the visits coming from email and Ad A
email_A = ad_clicks[(ad_clicks.utm_source == 'email') & (ad_clicks.experimental_group == 'A')]
print(email_A.head(3))

# see the visits coming from email and Ad A on Saturday or Sundays 
email_AS = email_A[email_A.day.isin(['Saturday', 'Sunday'])]
print(email_AS.head(3))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
                                user_id utm_source       day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
                                 user_id utm_source      day  \
8   018cea61-19ea-4119-895b-1a4309ccb148      email   Monday   
9   01

In [25]:
# How many views came from each platfrom?
visit_platform = ad_clicks.groupby('utm_source').user_id.count().reset_index()
# Rename the columns to be more representative
visit_platform.columns = ['platform', 'nr_of_views']
print(visit_platform)

   platform  nr_of_views
0     email          255
1  facebook          504
2    google          680
3   twitter          215


In [31]:
# How many visits come from each platform separated based on the weekday and weekend

# Add a column for weekday and weekend 
ad_clicks['weekday'] = ad_clicks.apply(lambda row: 'No' if (row['day'] == 'Saturday') | (row['day'] == 'Sunday') else 'Yes', axis=1)
print(ad_clicks.head(3))

visit_platform_w= ad_clicks.groupby(['utm_source', 'weekday']).user_id.count().reset_index()
print(visit_platform_w)



                                user_id utm_source       day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   Tuesday   

  ad_click_timestamp experimental_group weekday  
0               7:18                  A      No  
1                NaN                  B      No  
2                NaN                  A     Yes  
  utm_source weekday  user_id
0      email      No       69
1      email     Yes      186
2   facebook      No      137
3   facebook     Yes      367
4     google      No      188
5     google     Yes      492
6    twitter      No       60
7    twitter     Yes      155


In [12]:
# Which platfrom drived the most visits?
max_view = visit_platfrom[visit_platfrom.nr_of_views == visit_platfrom.nr_of_views.max()]
print(max_view)

  platfrom  nr_of_views
2   google          680


In [13]:
#Add a column to identify clicks 
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks.head(3))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  


In [15]:
# The percent of real people who clicked on ads from each platform
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
# print(clicks_by_source)
# Pivot the data
clicks_pivot = clicks_by_source.pivot(columns = 'is_click', index = 'utm_source', values='user_id').reset_index()
print(clicks_pivot)
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66
is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


In [16]:
# A versus B ads, display and click 
A_B = ad_clicks.groupby('experimental_group').user_id.count()
print(A_B)
A_B_actual = ad_clicks.groupby(['experimental_group','is_click']).user_id.count()
print(A_B_actual)

experimental_group
A    827
B    827
Name: user_id, dtype: int64
experimental_group  is_click
A                   False       517
                    True        310
B                   False       572
                    True        255
Name: user_id, dtype: int64


In [20]:
# clicks for each ad type A and B
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
print(a_clicks.head(2))
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']
print(b_clicks.head(2))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
2                NaN                  A     False  
                                user_id utm_source          day  \
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook   7 - Sunday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google  2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
1                NaN                  B     False  
3                NaN                  B     False  


In [27]:
# clicks % per day for each ad type A and B
a_clicks_n = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
#print(a_clicks_n)
a_clicks_nn = a_clicks_n.pivot(columns='is_click', index='day', values='user_id').reset_index()
a_clicks_nn['day%'] = a_clicks_nn[True] / (a_clicks_nn[True] + a_clicks_nn[False])

b_clicks_n = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
#print(b_clicks_n)
b_clicks_nn = b_clicks_n.pivot(columns='is_click', index='day', values='user_id').reset_index()
b_clicks_nn['day%'] = b_clicks_nn[True] / (b_clicks_nn[True] + b_clicks_nn[False])

print("% of the clicks per day for Ad A")
print(a_clicks_nn)
print(" ")
print("% of the clicks per day for Ad B")
print(b_clicks_nn)
print(" ")
print("It's clear Ad A had highest click through ratio")

% of the clicks per day for Ad A
is_click            day  False  True      day%
0            1 - Monday     70    43  0.380531
1           2 - Tuesday     76    43  0.361345
2         3 - Wednesday     86    38  0.306452
3          4 - Thursday     69    47  0.405172
4            5 - Friday     77    51  0.398438
5          6 - Saturday     73    45  0.381356
6            7 - Sunday     66    43  0.394495
 
% of the clicks per day for Ad B
is_click            day  False  True      day%
0            1 - Monday     81    32  0.283186
1           2 - Tuesday     74    45  0.378151
2         3 - Wednesday     89    35  0.282258
3          4 - Thursday     87    29  0.250000
4            5 - Friday     90    38  0.296875
5          6 - Saturday     76    42  0.355932
6            7 - Sunday     75    34  0.311927
 
It's clear Ad A had highest click through ratio
